In [4]:
using Revise

using RigidBodyDynamics
using RigidBodyDynamics: Bounds

using Plots
using BilevelTrajOpt

In [41]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)
body = findbody(mechanism, "ball")
basejoint = joint_to_parent(body, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)
position_bounds(floatingjoint) .= Bounds(-100, 100)
velocity_bounds(floatingjoint) .= Bounds(-100, 100)
μ = 0.5
motion_type = :xyz
point = Point3D(default_frame(findbody(mechanism, "floor")), [0.,0.,0.])
normal = FreeVector3D(default_frame(findbody(mechanism, "floor")), [0.,0.,1.])
hs = HalfSpace(point, normal)
floor = Obstacle(hs, μ, motion_type)
obstacles = [floor]
env = parse_contacts(mechanism, urdf, obstacles)
x0 = MechanismState(mechanism)
Δt = 0.005
sim_data = get_sim_data(x0,env,Δt,true);

In [42]:
q0 = [1., 0., 0., 0., 0., 0., 0.]
v0 = [0., 0., 0., 0.2, 5., 0.]

u0 = zeros(sim_data.num_v)
set_configuration!(x0, q0)
set_velocity!(x0, v0)
setdirty!(x0)
ctrl! = (u,t,x) -> u[:] = 0.
traj = BilevelTrajOpt.simulate(x0,env,sim_data.Δt,1,ctrl!,implicit_contact=false)
qnext = traj[1:sim_data.num_q,2]
vnext = traj[sim_data.num_q+1:sim_data.num_q+sim_data.num_v,2]
x_sol_exp = traj[sim_data.num_q+sim_data.num_v+2:end,2]

# qnext = q0
# vnext = v0

τ_ip, x_sol_ip, λ_ip, μ_ip, fx_ip = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=true);

τ_auglag, x_sol_auglag, λ_auglag, μ_auglag, fx_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=false);

# display("Explicit")
# display(x_sol_exp)
display("IP")
display(x_sol_ip)
display("Aug Lag")
display(x_sol_auglag)

display("torques")
display(τ_ip)
display(τ_auglag)

"IP"

6-element Array{Float64,1}:
 0.0    
 0.0    
 0.0    
 0.5    
 4.97547
 1.0    

"Aug Lag"

6-element Array{Float64,1}:
 0.00943759
 0.00486198
 0.00943759
 0.495318  
 0.00693041
 0.999305  

"torques"

6-element Array{Float64,1}:
  0.0        
  0.0        
  0.0        
  9.01034e-16
  4.905      
 -9.81       

6-element Array{Float64,1}:
  0.0        
  0.0        
  0.0        
  4.84903e-10
  4.81137    
 -9.80318    

Solve_Succeeded
Solve_Succeeded


In [43]:
# autodiff 
J_auto = ForwardDiff.jacobian(qv -> solve_implicit_contact_τ(sim_data,q0,v0,u0,qv[1:sim_data.num_q],qv[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)[2], vcat(qnext,vnext))

# numerical
ϵ = 1e-16
J_num = zeros(size(J_auto))
qv = vcat(qnext,vnext)
for i = 1:length(qv)
    δ = zeros(length(qv))
    δ[i] = ϵ
    J_num[:,i] = (solve_implicit_contact_τ(sim_data,q0,v0,u0,(qv+δ)[1:sim_data.num_q],(qv+δ)[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)[2] - x_sol_auglag)/ϵ
end

display(J_auto)
display(J_num)

6×13 Array{Float64,2}:
 0.0  -0.0695743     4.00962e-6   …    2.83657       1.41839    
 0.0  -0.053865      1.95535e-10       2.19643       1.09814    
 0.0  -0.0695743    -4.01019e-6        2.83657       1.41839    
 0.0  -0.137863      5.00504e-10     -14.7653        2.81087    
 0.0   2.87513e-13  -5.50277e-13      -3.80409e-6   -1.77797e-30
 0.0  -1.0           1.58926e-10  …    0.000126145  20.3874     

6×13 Array{Float64,2}:
 0.0  0.0  -1.2546e8   -1.51222e11  0.0  …  -3.80738e7   9.85932e11  0.0  0.0
 0.0  0.0   7.90836e7  -1.18577e12  0.0      2.39998e7  -1.54022e12  0.0  0.0
 0.0  0.0  -1.2546e8   -1.51222e11  0.0     -3.80738e7   9.85932e11  0.0  0.0
 0.0  0.0   2.0539e8   -3.10925e12  0.0      6.23305e7  -4.05044e12  0.0  0.0
 0.0  0.0   0.0         1.34648e10  0.0      0.0         1.8919e9    0.0  0.0
 0.0  0.0   0.0        -1.34277e13  0.0  …   0.0        -9.76563e12  0.0  0.0

In [10]:
maximum(abs.(J_auto-J_num))

0.00011239110444094536